# Clase 21: Reinforcement Learning

**MDS7202: Laboratorio de Programación Científica para Ciencia de Datos**

## Objetivos

- Introducir al estudiante a la disciplina de Reinforcement Learning
- Conocer sobre MDP y sus principales componentes
- Aprender a resolver problemas de RL mediante Q-learning
- Aprender a implementar algoritmos basados en DRL usando Stable-Baselines3

## Sistemas Autónomos

Los **Sistemas Autónomos** son modelos o sistemas diseñados para operar sin la necesidad de supervisión humana. Al igual que otros sistemas, reciben una entrada $X$ y generan una salida $f(X)$. Sin embargo, su característica distintiva es la capacidad de operar de manera independiente. Esta autonomía se basa en la capacidad del sistema para ejecutar acciones en momentos específicos, conocidos como **triggers**. Por ejemplo, un sistema autónomo podría activarse automáticamente una vez al día a las 12:00 PM, o en respuesta a acciones específicas del usuario, como enviar un mensaje a un chatbot.

El siguiente diagrama ilustra la idea del párrafo anterior:

<div style="text-align: center;">
    <img src="../../recursos/2024-01/rl/autonomous.png" style="width: 35%;">
</div>

Veamos un ejemplo de un sistema autónomo simple:

In [ ]:
import numpy as np

# definición de modelo
f = lambda x: 1 if x >= 0 else -1 # modelo

# modelo en "producción"
outputs = []
for _ in range(1000): # período en el que se ejecuta el sistema
  x = np.random.randn() # entrada del sistema
  f_x = f(x) # salida del sistema
  outputs.append(f_x) # guardamos salida

outputs[:10]

A lo largo de esta y la próxima clase trabajaremos con dos grandes paradigmas para generar sistemas autónomos: **Reinforcement Learning** y **LLM**.

## Reinforcement Learning

**Reinforcement Learning** (también conocido como aprendizaje reforzado) es un marco para resolver tareas de control (también llamadas problemas de decisión). El concepto básico de esta disciplina es la creación de **agentes** que aprenden del **ambiente** interactuando con él mediante **acciones** a través de ensayo y error, y recibiendo **recompensas** (positivas o negativas) como retroalimentación única. El aprendizaje por refuerzo es uno de los tres paradigmas básicos del aprendizaje automático, junto con el aprendizaje supervisado y el aprendizaje no supervisado.

El aprendizaje por refuerzo se diferencia del aprendizaje supervisado en que **no necesita pares de entrada/salida etiquetados**, ni necesita que las acciones subóptimas sean corregidas explícitamente. En su lugar, se enfoca en encontrar un equilibrio entre el **tradeoff exploración-explotación** (gastar "tiempo" en adquirir nuevo conocimiento vs gastarlo en hacer uso de lo que ya sabe) con el objetivo de **maximizar la recompensa** a largo plazo.

El entorno se suele describir en forma de un **Proceso de Decisión de Markov** (MDP, por sus siglas en inglés), ya que muchos algoritmos de aprendizaje por refuerzo para este contexto utilizan técnicas de programación dinámica. La principal diferencia entre las técnicas clásicas y los algoritmos de aprendizaje por refuerzo es que para estos últimos no es necesario el conocimiento de los MDP y se dirigen a grandes MDP donde los métodos exactos se convierten en no viables.

Entre algunos ejemplos notables de aplicaciones de RL podemos encontrar:
- [Robótica](https://www.youtube.com/watch?v=k7K7JG_RQhQ)
- [Vehículos Autónomos ](https://www.youtube.com/watch?v=fKBes088KV4)
- [AlphaGo](https://deepmind.google/technologies/alphago/)

<div style="text-align: center;">
    <img src="https://datasciencedojo.com/wp-content/uploads/ml-ds-algos.jpg" style="width: 50%;">
</div>

## Problema de hoy

Para ejemplificar los conceptos y consolidar lo aprendido, el dia de hoy trabajaremos en resolver el videojuego **Frozen Lake**. Este videojuego ilustra la tarea de un elfo navideño, el cual debe atravesar por un lago congelado evitando caer al agua y así recuperar el regalo de navidad. Pueden encontrar la definición formal del problema en este [link](https://gymnasium.farama.org/environments/toy_text/frozen_lake/).

Conociendo el contexto, **¿como podemos usar RL para resolver este problema?**

<div style="text-align: center;">
    <img src="https://www.gymlibrary.dev/_images/frozen_lake.gif" style="width: 20%;">
</div>

## Markov Decision Process (MDP)

El primer paso para usar RL es formular nuestro problema como un **Proceso de Decisión de Markov** (MDP). En matemáticas, un MDP es un proceso de control estocástico en tiempo discreto, el cual proporciona un marco matemático para **modelar la toma de decisiones** en situaciones donde los resultados son en parte aleatorios y en parte controlados por un tomador de decisiones. Los MDP son útiles para estudiar problemas de optimización resueltos mediante programación dinámica. Los procesos de decisión de Markov son una extensión de las cadenas de Markov y son utilizados en muchas disciplinas, incluyendo la robótica, el control automático, la economía y la manufactura.

Un Proceso de Decisión de Markov se constituye de los siguientes elementos:

- **Estados**: Información que obtiene el agente del ambiente 
- **Acciones**: Set de acciones posibles a realizar por el agente
- **Recompensas**: Feedback otorgado al agente por ejecutar una acción $a$ en un estado $s$

donde la colección $(S_0, A_0, R_1, S_1)$ se le conoce como una **experiencia**.

<div style="text-align: center;">
    <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/sars.jpg" style="width: 30%;">
</div>

De esta manera, en el tiempo $t$ el agente recibe el estado $S_t$, a lo que responde ejecutando una acción $A_t$. Esta acción altera el ambiente, otorgando una recompensa $R_t$ y retornando un nuevo estado $S_{t+1}$. Dado que $S_{t+1}$ depende únicamente del conjunto $(S_t, A_t)$ se concluye que la transición de entre estados cumple la **Propiedad de Markov**: Agentes solo deben tener en cuenta el **estado actual** $S_t$ para decidir qué acción tomar y **<u>no</u> la historia de todos los estados y acciones anteriores**.

El siguiente diagrama ilustra la representación típica de un MDP:

<div style="text-align: center;">
    <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/RL_process.jpg" style="width: 50%;">
</div>

Detallemos un poco mejor cada uno de los componentes antes mencionados.

### Estados

Las **Observaciones o Estados** son la información que nuestro agente obtiene del entorno. En el caso de un videojuego, esto podría ser una captura de pantalla. Para un agente de trading, podría ser el valor de una acción específica, entre otros ejemplos.

Sin embargo, hay una diferencia que hacer entre observación y estado:

- **Estado**: Entrega una **descripción completa** del estado del ambiente. Es un entorno completamente observado. 
- **Observation**: Es una **descripción parcial** del estado del ambiente.

<div style="text-align: center;">
    <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/obs_space_recap.jpg" style="width: 35%;">
</div>

Un aspecto importante a considerar es que **no existe una única representación de las observaciones.** Por ejemplo, el MDP puede recoger observaciones de solo algunas piezas del ajedrez y aún así llegar a un buen resultado (aunque esto probablemente sea una mala idea). Es responsabilidad del programador preocuparse para que las observaciones recibidas por el agente sean de utilidad para obtener buenos resultados, teniendo especial cuidado en respetar la Propiedad de Markov.

Volviendo al problema de **Frozen Lake**, los estados son definidos a partir de la posición en la cuadrídula del agente (elfo). Como el mapa tiene un tamaño de 4x4, existen 16 estados posibles, los cuales pueden ser enumerados de la siguiente manera:

<div style="text-align: center;">
    <img src="../../recursos/2024-01/rl/states.png" style="width: 20%;">
</div>

Donde se observa que el agente comenzará siempre en el estado 0 y su meta es llegar al estado 15.

### Acciones

Las acciones hacen referencia a todo el set posible de acciones a ejecutar por el agente. 

El espacio de acciones puede ser de dos tipos:

- **Discreto**: El número de acciones posibles es **finito**
- **Continuo**: El número de acciones posibles es **infinito**

<div style="text-align: center;">
    <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit1/action_space.jpg" style="width: 35%;">
</div>

**Ojo! Tener presente el tipo de acciones (discretas o continuas) de nuestro ambiente es un aspecto <u>crucial</u> para elegir el algoritmo de RL.**

Retornando a nuestro problema, **Frozen Lake** puede ser caracterizado como un ambiente de **acciones discretas** con 4 acciones posibles:

- 0: Izquierda ⬅️
- 1: Abajo ⬇️
- 2: Derecha ➡️
- 3: Arriba ⬆️

donde cada elemento representa la dirección de movimiento del elfo navideño.

### Recompensas

Las recompensas son fundamentas en RL pues son el **único feedback del agente**. A través del agente, el **agente aprende si la acción ejecutada fue buena o no**. 

En el ejemplo del ajedrez, el agente puede obtener una recompensa positiva si captura una pieza del oponente, y una recompensa negativa si el oponente captura una pieza del agente.

La recompensa acumulada en cada timestep $t$ puede ser escrita como:

$$R(\tau) = r_{t+1} + r_{t+2} + r_{t+3} + r_{t+4} + ...$$
$$R(\tau) = \sum_{k=0}^{\infty} r_{t+k+1}$$

Dado que las recompensas futuras son más lejanas de $t$ y por lo tanto menos deseables, un procedimiento común es descontar las recompensas futuras por un factor $\gamma$:

$$R(\tau) = r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + \gamma^3 r_{t+4} + ...$$
$$R(\tau) = \sum_{k=0}^{\infty} \gamma^k r_{t+k+1}$$

donde $\gamma$ es un factor definido por el programador y usualmente varia entre los valores 0.95 y 0.99.

Al igual que los estados, el **esquema de recompensas no es único y puede ser configurable por el usuario**. Es responsabilidad del usuario definir un buen esquema de recompensas acorde al problema a resolver para lograr una solución efectiva.

Para el problema de **Frozen Lake**, las recompensas son definidas en función de la posición del elfo:

- Regalo: +1
- Agua: +0
- Hielo: +0

> **Pregunta**: ¿Qué problemas podría tener este esquema de recompensas?

## Formalización del problema

El objetivo de Aprendizaje Reforzado es **encontrar una política $\pi$ que maximice la recompensa del agente** a lo largo del experimento. La política $\pi$ puede ser interpretada como una función que, dado un estado $s$, retorna la **probabilidad** de tomar cada acción disponible a ejecutar por el agente.

<div style="text-align: center;">
    <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/policy.jpg" style="width: 35%;">
</div>

En otras palabras, el problema de optimización a resolver puede ser formalizado por:


$$\max_\pi J(\pi) = \mathbb{E}_{\tau \sim p_\pi(\tau)}\left[\sum_{t=1}^T \gamma^{t-1} r\left(s_t, a_t \sim \pi (a|s)\right)\right]$$


Existen 2 grandes familias de métodos para encontrar $\pi$: **Value-Based** y **Policy-Based**. Revisemos en detalle cada uno de estos.

## Métodos RL

### Value-Based

En los métodos **Value-Based**, el objetivo es **aprender y representar la función de valor de un estado y acción**, denotada comúnmente como $Q(s,a)$. La función de valor refleja la **utilidad esperada asociada de tomar una acción $a$ en un estado $s$** en términos de la recompensa acumulada. La ecuación de Bellman para $Q(s, a)$ en términos de la recompensa inmediata y la función de valor usando el estado siguiente $s'$ es una expresión clave en estos métodos:

$$Q(s, a) = R(s, a) + \gamma \max_a Q(s', a)$$

Donde $s$ y $a$ representan el estado actual y la acción tomada, respectivamente, $s'$ es el siguiente estado, $\gamma$ es el factor de descuento que pondera las recompensas futuras, y $\max_a Q(s', a)$ denota la estimación del valor máximo del próximo estado. Si bien existen algunas excepciones, los algoritmos Value-Based son **generalmente de acciones discretas**. Algunos algoritmos notables de este enfoque son Q-learning y Deep Q-Network.

Finalmente con la representación del valor de cada par $(s, a)$, la política $\pi$ puede ser formulada como:

$$\pi(s) = \arg\max_{a} Q(s, a)$$

### Policy-Based

En contraste, los métodos **Policy-Based** buscan aprender directamente la política de decisión del agente, es decir, la distribución de probabilidad sobre las acciones condicionadas al estado. La política es denotada como $\pi(a|s, \theta)$, que representa la probabilidad de elegir la acción $a$ dado el estado $s$ y parámetros $\theta$. La actualización de la política se realiza mediante gradientes, y la ecuación de la política es fundamental en este enfoque:

$$\nabla_\theta J(\theta) \propto \sum_s \mu(s) \sum_a \nabla_\theta \pi(a|s, \theta)Q^\pi(s, a)$$

Donde $J(\theta)$ representa el rendimiento esperado de la política, $\mu(s)$ es la distribución estacionaria de estados bajo la política actual, $\pi(a|s, \theta)$ es la política parametrizada por $\theta$ y $Q^{\pi}(s,a)$ es la función de valor bajo la política $\pi$. Si bien existen algunas excepciones, los algoritmos Policy-Based son **generalmente de acciones continuas**. Ejemplos notables de este enfoque son los algoritmos REINFORCE y TRPO.

### Resumen de Métodos

En resumen, existen 2 familias de métodos:

- **Policy-Based**: Entrenar la política directamente para aprender qué accion tomar dado un estado $s$
- **Value-Based**: Entrenar una función de valor para aprender qué estado tiene mayor valor. Usar esta función para tomar la acción que nos lleve al mejor estado. 

<div style="text-align: center;">
    <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/two-approaches.jpg" style="width: 35%;">
</div>

## Q-learning

**Q-learning** es un algoritmo de reinforcement learning del tipo **Value-Based** (es decir, tiene por objetivos aprender el valor de una acción $a$ en un determinado estado $s$). La principal diferencia y virtud de este algoritmo es su simplicidad, pues intenta modelar la representación $Q(s, a)$ de forma **tabular**. De esta forma, este algoritmo permite a un agente tomar decisiones óptimas mediante la actualización iterativa de los valores tabuleras basados en las recompensas obtenidas al interactuar con el entorno.

### ¿Cómo funciona?

El algoritmo puede explicar en los siguientes pasos:

1. **Inicializar la tabla Q**: Crear e inicializar una matriz de dimensión para cada par $(s,a)$, por ejemplo:

<style>
  table {
    margin: auto;
  }
</style>

|          | Izquierda | Abajo | Derecha | Arriba |
|----------|-----------|-------|---------|--------|
| Estado 1   | 0         | 0     | 0       | 0      |
| Estado 2   | 0         | 0     | 0       | 0      |
| Estado 3   | 0         | 0     | 0       | 0      |
| Estado 4   | 0         | 0     | 0       | 0      |

<br>

> **Pregunta**: ¿Qué desventajas podría tener este enfoque?

2. **Recolectar experiencias**: A partir de la interacción con el ambiente, recolectar experiencias:

$$(S_t, A_t, R_{t+1}, S_{t+1})$$

3. **Actualizar la tabla Q**: Utilizando la [Ecuación de Bellman](https://huggingface.co/learn/deep-rl-course/unit2/bellman-equation), actualizar los valores de $Q(s,a)$ usando la siguiente expresión:

<div style="text-align: center;">
    <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-8.jpg" style="width: 40%;">
</div>

### Epsilon-greedy

En las primeras secciones de esta clase hablamos del **trade-off entre exploración y explotación**, es decir, el balance que debe realizar nuestro agente para (i) obtener nuevo conocimiento y (ii) hacer uso de su conocimiento actual para obtener el máximo de recompensa. Para resolver este dilema, Q-learning utiliza la estrategia **epsilon-greedy**:

\begin{equation}
a_t = 
\begin{cases} 
\text{random action}, & \text{with probability } \epsilon_t \\
\underset{a}{\mathrm{argmax}} \, Q(S_t, a), & \text{with probability } 1 - \epsilon_t
\end{cases}
\end{equation}

De esta manera, con probabilidad $\epsilon_t$ el agente toma una acción aleatoria (es decir, *explora* nuevas acciones y estados), mientras que con probabilidad $1 - \epsilon_t$ el agente elige el par $(s,a)$ con la mayor función de valor (el agente "rentabiliza" su conocimiento).

Noten que como la política epsilon-greedy es una política de acciones diferente a la política óptima $\pi$, se dice que el algoritmo Q-learning es del tipo **off-policy** (es decir, utiliza una política distinta para tomar acciones y actualizar su aprendizaje). 

Por último, algo común es imponer que la fase de exploración sea intensiva en la primera parte del entrenamiento, para luego dedicarse a explotar el conocimiento adquirido. En otras palabras, es deseable que la probabilidad de exploración $\epsilon_t$ disminuya mientras más se entrene el agente. Esto se puede lograr agregando un factor **decay** a epsilon, el cual puede ser lineal, exponencial, etc. Veremos una implementación de este último un poco mas adelante.

<div style="text-align: center;">
    <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-5.jpg" style="width: 30%;">
</div>

### Pseudocódigo

En función de todo lo aprendido hasta ahora, el algoritmo Q-learning puede escribirse usando el siguiente pseudocódigo:

<div style="text-align: center;">
    <img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-2.jpg" style="width: 50%;">
</div>

## Implementación: Resolviendo FrozenLake con Q-learning

### Instalar librerias necesarias

Como siempre, antes de trabajar debemos instalar las librerias necesarias:

In [ ]:
# descomentar estas lineas para instalar librerias
#!pip install swig
#!pip install gymnasium
#!pip install gymnasium[box2d]
#!pip install stable_baselines3

### Inicializar el ambiente FrozenLake

Para trabajar con el ambiente **FrozenLake** haremos uso de la libreria [gymnasium](https://gymnasium.farama.org). Esta libreria brilla por tener múltiples implementaciones de problemáticas de RL en versión MDP, los que podemos utilizar con unas pocas lineas de código.

Veamos como podemos hacer uso de esta librería para nuestro problema:

In [ ]:
import gymnasium as gym

env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False)
env

Cada ambiente de `gym` tiene 3 métodos por default:
- **reset**: Reinicia el ambiente a su estado inicial y devuelve el estado inicial junto a información adicional. Para el caso particular de FrozenLake, devuelve el elfo a la posición 0.
- **step**: Recibe una acción del agente, devuelve:
    - **new_state**: El nuevo estado del ambiente ($S_{t+1}$)
    - **reward**: La recompensa generada por el ambiente ($R_{t+1}$)
    - **terminated**: Booleano si el ambiente llegó a su estado terminal
    - **truncated**: Similar a terminated, Booleano si el ambiente llegó a un estado truncado
    - **info**: Información adicional para debuggear
- `close`: Cierra el ambiente

In [ ]:
env.reset()

In [ ]:
env.step(1)

In [ ]:
env.close()

Cada ambiente tiene además dos atributos muy importantes: `action_space` y `observation_space`. Estos atributos definen el espacio posible de acciones y observaciones, respectivamente:

In [ ]:
env.action_space

In [ ]:
env.observation_space

En este caso, el ambiente permite 4 acciones discretas y 16 estados discretos. 

Finalmente, algo útil para nuestro algoritmo es guardar la dimensionalidad de espacios:

In [ ]:
dim_actions = env.action_space.n # dimensión de las acciones
dim_states = env.observation_space.n # dimensión de los estados (observaciones)

print('dim_actions:', dim_actions)
print('dim_states:', dim_states)

### Implementación Q-learning

Con lo apredido en la sección anterior, podemos implementar Q-learning de forma nativa:

In [ ]:
import random
import numpy as np

class QAgent():
  def __init__(self, dim_states, dim_actions, gamma = 0.95, lr = 2e-1, epsilon = 0.99, decay = 1e-4):
    '''
    initialize QAgent parameters
    '''
    self.dim_states = dim_states
    self.dim_actions = dim_actions
    self.lr = lr
    self.epsilon = epsilon
    self.gamma = gamma
    self.decay = decay

    # initialize Q-table
    self.Qtable = np.zeros((dim_states, dim_actions))

  def select_action(self, state, greedy=False):
    '''
    select action using epsilon-greedy
    '''
    # randomly generate a number between 0 and 1
    random_int = random.uniform(0, 1)
    if random_int > self.epsilon or greedy:  # exploitation
        action = np.argmax(self.Qtable[state])  # take the action with the highest value given a state
    else:  # exploration
        action = np.random.randint(0, self.dim_actions)  # take random action
        self.epsilon *= (1 - self.decay)

    return action

  def update(self, state, action, reward, new_state, done):
    '''
    update Q-table
    '''
    self.Qtable[state][action] += self.lr * (reward + self.gamma * np.max(self.Qtable[new_state, :]) * (1 - done) - self.Qtable[state, action])

Probemos ahora el funcionamiento de nuestro agente:

In [ ]:
agent = QAgent(dim_states, dim_actions) # noten como el agente debe recibir (dim_states, dim_actions) para inicializar la tabla
agent

In [ ]:
agent.select_action(1)

### Loop de entrenamiento

Con el ambiente y el agente, podemos pasar a entrenar el agente:

In [ ]:
# hyperparams
episodes = 1000
timesteps = 100

# training loop
for episode in range(episodes):
  # reset env
  state, info = env.reset() 
  for timestep in range(timesteps):
    # select action
    action = agent.select_action(state)
    # obtain env response
    new_state, reward, done, truncated, info = env.step(action)
    # gather experience
    experience = (state, action, reward, new_state, done)
    # update agent parameters
    agent.update(*experience)
    # stop if done
    if done or truncated:
      break
    # update state
    state = new_state

Nuestro código funcionó correctamente, pero como sabemos si nuestro agente efectivamente pudo resolver FrozenLake?

### Evaluación

Para evaluar que nuestro agente converge a una política óptima, haremos uso de 2 métodos de evaluación:

1. **Cuantificar recompensas en entrenamiento**: La idea es medir el nivel de recompensas promedio cada X steps de entrenamiento
2. **Evaluación visual**: Observar al agente jugando al juego FrozenLake

In [ ]:
def test_agent(env, agent, test_episodes = 20, test_timesteps = 100):
  '''
  returns avg and std of rewards
  '''
  test_rewards = np.zeros(test_episodes)
  for episode in range(test_episodes):
    episode_reward = 0
    state, info = env.reset()
    for timestep in range(test_timesteps):
      action = agent.select_action(state, greedy = True) # notar como se evitan las acciones de exploración
      new_state, reward, done, truncated, info = env.step(action)
      episode_reward += reward
      if done:
        break
      state = new_state
    test_rewards[episode] = episode_reward

  return np.mean(test_rewards), np.std(test_rewards)

In [ ]:
import gymnasium as gym

# init env
env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False)

# gather (dim_actions, dim_states)
dim_actions = env.action_space.n
dim_states = env.observation_space.n

# init agent
agent = QAgent(dim_states, dim_actions)

# set hyperparams
episodes = 1000
timesteps = 100
test_each = 20

# init arrays for metrics
test_episodes, test_avg_rewards, test_std_rewards = [], [], []

# training loop
for episode in range(episodes):
  # reset env
  state, info = env.reset() 
  for timestep in range(timesteps):
    # select action
    action = agent.select_action(state)
    # obtain env response
    new_state, reward, done, truncated, info = env.step(action)
    # gather experience
    experience = (state, action, reward, new_state, done)
    # update agent parameters
    agent.update(*experience)
    # stop if done
    if done or truncated:
      break
    # update state
    state = new_state

  # test each X episodes
  if episode % test_each == 0:
    avg_rewards, std_rewards = test_agent(env, agent)
    test_episodes.append(episode)
    test_avg_rewards.append(avg_rewards)
    test_std_rewards.append(std_rewards)

Veamos ahora las recompensas obtenidas por el agente a lo largo del entrenamiento:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (12, 4))
plt.errorbar(test_episodes, test_avg_rewards, test_std_rewards, marker = '.')
plt.xlabel('Episodes')
plt.ylabel('Test Rewards')
plt.show()

Para evaluar el comportamiento del agente de forma visual, podemos inicializar el ambiente con el argumento `render_mode="human"`. Esta opción no funciona bien en Google Colab, por lo que les recomiendo correr este código desde sus equipos personales.

Veamos primero como se ve un agente que toma **acciones aleatorias**:

In [ ]:
env = gym.make("FrozenLake-v1", render_mode="human", map_name="4x4", is_slippery=False)

for episode in range(10):
    done = truncated = False
    state, info = env.reset()
    for timestep in range(100):
        action = env.action_space.sample()  # agent policy that uses the observation and info
        state, reward, done, truncated, info = env.step(action)
        if done or truncated:
            break

env.close()

Luego, veamos como se ve el agente que entrenamos con Q-learning:

In [ ]:
env = gym.make("FrozenLake-v1", render_mode="human", map_name="4x4", is_slippery=False)

for episode in range(10):
    done = truncated = False
    state, info = env.reset()
    for timestep in range(100):
        action = agent.select_action(state, greedy = True)
        action = action.item()
        state, reward, done, truncated, info = env.step(action)
        if done or truncated:
            break

env.close()

## Stable-Baselines3

En la sección anterior implementamos de forma nativa el algoritmo Q-learning para resolver FrozenLake. Si bien tuvimos resultados satisfactorios, implementar nativamente un algoritmo no es lo más deseable para trabajar de forma profesional, especialmente en el campo de RL donde los algoritmos actuales son **diametralmente de mayor complejidad**.

[`Stable-Baselines3`](https://stable-baselines3.readthedocs.io/en/master/) es una librería que nos facilita la implementación de algoritmos de RL en pocas líneas de código. Gracias a su amplia aceptación y a su equipo de mantenimiento, esta librería se ha posicionado como una de las principales alternativas para resolver problemas de RL, pudiendo hacer uso de sus modelos en pocas líneas de código. 

A diferencia de librerías como scikit-learn, SB3 provee la mayor parte de su soporte a la implementación de modelos *deep*, es decir, modelos que usan **redes neuronales** por detrás. De esta forma, no sólo podemos implementar modelos de RL en pocas líneas de código, sino que estos modelos además son bastantes cercanos al **estado del arte**.

<div style="text-align: center;">
    <img src="https://stable-baselines3.readthedocs.io/en/master/_static/logo.png" style="width: 30%;">
</div>

Veamos como hacer uso de `SB3` para resolver el problema FrozenLake usando DQN (Q-learning pero con redes neuronales):

In [ ]:
import gymnasium as gym
from stable_baselines3 import DQN

# Create environment
env = gym.make("FrozenLake-v1", render_mode="rgb_array", map_name="4x4", is_slippery=False)

# init agent
model = DQN("MlpPolicy", env, verbose=0)
# train the agent and display a progress bar
model.learn(total_timesteps=int(2e5), progress_bar=True)

Noten como para entrenar el agente no tuvimos que escribir el loop de entrenamiento. 

**Ojo! Como estos modelos son de mayor complejidad, en algunos casos el entrenamiento puede tardar bastante. Pueden acelerar exponencialmente el entrenamiento de su modelo ejecutando el código en Google Colab y habilitando la opción de GPU.**

Luego, pueden exportar el modelo entrenado a un archivo `.zip` en pocas lineas de código:

In [ ]:
model.save("dqn_lunar") # export model (i.e from Google Colab)
del model  # delete trained model to demonstrate loading
model = DQN.load("dqn_lunar", env=env) # load model

Pasemos ahora a evaluar nuestro agente usando la función `evaluate_policy`. Al igual que en la sección anterior, esta función nos devuelve la recompensa promedio y su desviación obtenida por el agente:

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

# Evaluate the agent
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)
mean_reward, std_reward

Finalmente, podemos evaluar de manera visual el comportamiento de nuestro agente usando el mismo esquema de código:

In [ ]:
env = gym.make("FrozenLake-v1", render_mode="human", map_name="4x4", is_slippery=False)

for episode in range(10):
    done = truncated = False
    state, info = env.reset()
    for timestep in range(100):
        action, _states = model.predict(state, deterministic=True)
        action = action.item() # important step: gather action of array
        state, reward, done, truncated, info = env.step(action)
        if done or truncated:
            break
env.close()

## Bibliografía

- Sutton, R. S., & Barto, A. G. (2018). *Reinforcement learning: An introduction*. MIT press.
- [RL Course by David Silver](https://www.youtube.com/watch?v=2pWv7GOvuf0&list=PLzuuYNsE1EZAXYR4FJ75jcJseBmo4KQ9-)
- [Deep RL Course - HuggingFace](https://huggingface.co/learn/deep-rl-course/unit2/introduction)